## NLP for disaster management (Flooding)

The purpose of this notebook is to act as a development environment for the FDL floods project. 

The project as completed by the ESA team surrounds the use of image data to create flood segmentation models which can be applied on a Movidius neural network stick.

Given the use of image data in this case we have made the decision to approach the problem of disaster management another way namely through the use of natural language processing to bucket tweets into categories so that they can be queried by individuals on the ground 

### Loading appropriate scripts

In [ ]:
\l nlp/init.q
\l ml/ml.q
\l ../code/fdl_disasters.q
\l ../code/token_save.p
.ml.loadfile`:init.q

### Loading required python Functionality

In [ ]:
npa:.p.import[`numpy]`:array
plt:.p.import[`matplotlib.pyplot]
pd :.p.import[`pandas]

// Textual data preprocessing 
token:.p.import[`keras.preprocessing.text]`:Tokenizer
pad  :.p.import[`keras.preprocessing.sequence]`:pad_sequences

kl:{.p.import[`keras.layers]x}
// Keras layers used
seq    :.p.import[`keras.models]`:Sequential
dense  :kl`:Dense
embed  :kl`:Embedding
lstm   :kl`:LSTM
spdrop1:kl`:SpatialDropout1D
dropout:kl`:Dropout

### Data loading and preprocessing 

In [ ]:
nms_1:{rename[;names]("j*SSS";enlist",")0:`$":../data/floods/",x}
nms_2:{("jSSIPSf*J*S";enlist",")0:`$":../data/floods/",x}

In [ ]:
/ Colorado
co :nms_1["Developed/2013_Colorado_floods/2013_Colorado_floods-tweets_labeled.csv"]
/ Queensland
qld:nms_1["Developed/2013_Queensland_floods/2013_Queensland_floods-tweets_labeled.csv"]
/ Alberta
alb:nms_1["Developed/2013_Alberta_floods/2013_Alberta_floods-tweets_labeled.csv"]
/ Phillipines
ph :nms_1["Developing/2012_Philipinnes_floods/2012_Philipinnes_floods-tweets_labeled.csv"]
/ India
ind:nms_2["Developing/2014_India_floods/2014_india_floods.csv"]
/ Pakistan
pac:nms_2["Developing/2014_Pakistan_floods/2014_pakistan_floods.csv"]

In [ ]:
// Label consolidation and joining of different format data
data_b:encodetgt[co,qld,alb,ph;encodebin],encodetgt[ind,pac;encodebin]
data_m:encodetgt[co,qld,alb,ph;encodemulti],encodetgt[ind,pac;encodemulti]
// Remove capitalization
data_m[`tweet_text]:data_b[`tweet_text]:lower data_m`tweet_text
// remove data from set which is missing labels
inds:exec i from data_m where target<>`not_labeled
data_b:data_b@inds
data_m:data_m@inds

In [ ]:
tweets:data_b
save`:tweets.txt

In [ ]:
// Textual data cleansing
rmv_list   :("http*";"rt";"@*";"*,";"*&*";"*[0-9]*")
rmv_single :rmv_master[;",.:?!/@'";""]
rmv_hashtag:rmv_master[;"#";""]

In [ ]:
5#data_m`tweet_text

In [ ]:
// Remove non ascii, retweet tag, numerical values, emojis and space hashtags 
data_m[`tweet_text]:data_b[`tweet_text]:
 (rmv_ascii rmv_custom[;rmv_list] rmv_hashtag rmv_single@) each data_m`tweet_text


In [ ]:
5#data_m`tweet_text

---

## Initial data analysis steps

The purpose of this section is to take a high level look at the data both in regards to the distribution of classes, explore what underlays the classes themselves and perform some rudimentary sentiment analysis on the data to find positive and negative tweets

---

### Word cloud analysis

In the following cells we display the most commonly associated words with a number of the classes within this dataset

In [ ]:
wordcloudfn[data_m;`affected_individuals]

In [ ]:
wordcloudfn[data_m;`donations_volunteering]

In [ ]:
wordcloudfn[data_m;`sympathy_prayers]

Given the context of the tweets that within the overall dataset it is not surprising that words such as flood and locations such as Pakistan and Kashmir are displayed within a number of the classes. However some displayed words are more informative to the classes,

* donations_volunteering:
    - affected, donate, rescue, volunteer, people
* affected_individuals:
    - killed, dead, affected, missing, toll
* sympathy_prayers:
    - pray, safe, friend, thought, love
    
The presence of distinct words within each class lends credence to the idea that it should be possible to classify these tweets into their associated categories and generalise this method to new and larger datasets

---

### Sentiment analysis

The following cells outline the use of the Kx NLP library to discern positive and negative sentiments from tweets within the corpus. Below we are displaying 5 of the 100 most positive and most negative tweets in the dataset.

This allows us to gain some understanding of the state of mind of those individuals affected by the crises

In [ ]:
// compute the sentiment of the tweets within the corpus
sentiments:.nlp.sentiment each data_m`tweet_text
\c 200 200
5?100#data_m[`tweet_text] iasc  sentiments`compound
5?100#data_m[`tweet_text] idesc sentiments`compound
\c 20 90

---

### Category distribution metrics

One important aspect of training machine learning models is an understanding of the distributions of categories within the datasets. This provides a pseudo benchmark from which we can assess the ability of the models to correctly categorize differing classes.

For example in an extremely skewed binary classification model with one class `1b` comprising 95% of the target labels a model which has an accuracy on the order of 95% would not be said to have 'learnt' how to distinguish the classes as guessing that all data was labeled `1b` it would achieve this accuracy.

As a result we look at other metrics to assess the model performance such as precision, recall and f1-score which are discussed within the whitepaper which details this work from a theoretical viewpoint

##### Binary Classification 

In [ ]:
distrib_b:desc count each group data_b`target

In [ ]:
plt[`:bar][til count distrib_b;value distrib_b;`color pykw `k];
plt[`:title][`$"Distribution of target categories within binary-class dataset"];
plt[`:xlabel][`Category];
plt[`:xticks][til count distrib_b;key distrib_b;`rotation pykw `45];
plt[`:ylabel][`$"#Tweets"];
plt[`:show][];

##### Multi-label Classification

In [ ]:
distrib_m:desc count each group data_m`target

In [ ]:
plt[`:bar][til count distrib_m;value distrib_m;`color pykw `g];
plt[`:title][`$"Distribution of target categories within multi-class dataset"];
plt[`:xlabel][`Category];
plt[`:xticks][til count distrib_m;key distrib_m;`rotation pykw `45];
plt[`:ylabel][`$"#Tweets"];
plt[`:show][];

---

### Model Binary-Class Disaster Management

In [ ]:
batch_sz    :64
embed_dim   :100
epochs      :10
max_nb_words:2000
max_seq_len :50

In [ ]:
py_data_b :npa data_b`tweet_text
tweet_vals:py_data_b

In [ ]:
tokenizer:token[`num_words pykw max_nb_words;`lower pykw 1b]
tokenizer[`:fit_on_texts]tweet_vals;

sv_tok:.p.get[`save_token]
sv_tok[tokenizer] // Save tokenizer to be used on 'live system'

In [ ]:
X_b:tokenizer[`:texts_to_sequences]tweet_vals
X_b:pad[X_b;`maxlen pykw max_seq_len]`

Y_b:flip value ohe:.ml.i.onehot1 data_b`target

In [ ]:
tts_b:.ml.traintestsplit[X_b;Y_b;0.1]
xtrn_b:tts_b`xtrain;ytrn_b:tts_b`ytrain
xtst_b:tts_b`xtest;ytst_b:tts_b`ytest

In [ ]:
mdl_b:seq[];
mdl_b[`:add][embed[max_nb_words;embed_dim;`input_length pykw (.ml.shape X_b)1]];
mdl_b[`:add]spdrop1[0.2];
mdl_b[`:add]lstm[50;pykwargs `dropout`recurrent_dropout!(0.2;0.2)];
mdl_b[`:add]dense[2;`activation pykw `softmax];
mdl_b[`:compile][pykwargs `loss`optimizer`metrics!(`categorical_crossentropy;`adam;enlist `accuracy)];
print mdl_b[`:summary][];

In [ ]:
strt:.z.t
mdl_b[`:fit][npa xtrn_b;npa ytrn_b;`epochs pykw epochs;`verbose pykw 0];
-1"\nTraining time = ",string .z.t-strt;

In [ ]:
class_scoring[xtst_b;ytst_b;mdl_b;ohe]

### Model Multi-Class Disaster Management

In [ ]:
py_data_m :npa data_b`tweet_text
tweet_vals:py_data_m
tokenizer:token[`num_words pykw max_nb_words;`lower pykw 1b]
tokenizer[`:fit_on_texts][tweet_vals];

In [ ]:
X_m:tokenizer[`:texts_to_sequences]tweet_vals
X_m:pad[X_m;`maxlen pykw max_seq_len]`

Y_m:flip value ohe:.ml.i.onehot1 data_m`target

In [ ]:
tts_m:.ml.traintestsplit[X_m;Y_m;0.1]
xtrn_m:tts_m`xtrain;ytrn_m:tts_m`ytrain
xtst_m:tts_m`xtest;ytst_m:tts_m`ytest

In [ ]:
mdl_m:seq[];
mdl_m[`:add][embed[max_nb_words;embed_dim;`input_length pykw (.ml.shape X_m)1]];
mdl_m[`:add]spdrop1[0.1];
mdl_m[`:add]lstm[100;pykwargs `dropout`recurrent_dropout!(0.1;0.1)];
mdl_m[`:add]dense[7;`activation pykw `softmax];
mdl_m[`:compile][pykwargs `loss`optimizer`metrics!(`categorical_crossentropy;`adam;enlist `accuracy)];
print mdl_m[`:summary][];

In [ ]:
strt:.z.t
mdl_m[`:fit][npa xtrn_m;npa ytrn_m;`epochs pykw epochs;`verbose pykw 0];
-1"\nTraining time = ",string .z.t-strt;

In [ ]:
class_scoring[xtst_m;ytst_m;mdl_m;ohe]

In [ ]:
mdl_m[`:save]["../live/multiclass_mdl.h5"]

---

### Balanced classes

Looking at the distribution of classes within the main dataset it is clear that the classes are imbalanced. To reduce biases which may exist as a result of this imbalance a balanced dataset was built and a model fit with the 'improved model' in order to test improvements that could be gained from the use of a more balanced dataset

In [ ]:
inds_m:raze{neg[x]?y}[min count each l]each value l:group data_m`target
bal_X_m:X_m@inds_m
bal_Y_m:Y_m@inds_m

In [ ]:
bal_tts_m:.ml.traintestsplit[bal_X_m;bal_Y_m;0.1]
xtrn_m:bal_tts_m`xtrain;ytrn_m:bal_tts_m`ytrain
xtst_m:bal_tts_m`xtest;ytst_m:bal_tts_m`ytest

In [ ]:
bal_mdl_m:seq[];
bal_mdl_m[`:add][embed[max_nb_words;embed_dim;`input_length pykw (.ml.shape X_m)1]];
bal_mdl_m[`:add]spdrop1[0.1];
bal_mdl_m[`:add]lstm[100;pykwargs `dropout`recurrent_dropout!(0.1;0.1)];
bal_mdl_m[`:add]dense[7;`activation pykw `softmax];
bal_mdl_m[`:compile][pykwargs `loss`optimizer`metrics!(`categorical_crossentropy;`adam;enlist `accuracy)];
print bal_mdl_m[`:summary][];

In [ ]:
strt:.z.t
bal_mdl_m[`:fit][npa xtrn_m;npa ytrn_m;`epochs pykw epochs;`verbose pykw 0];
-1"\nTraining time = ",string .z.t-strt;

In [ ]:
class_scoring[xtst_m;ytst_m;bal_mdl_m;ohe]

---